In [1]:
import pickle
# import graphvite as gv
import dataset
import nltk
import re
import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')



%reload_ext autoreload
%autoreload

[nltk_data] Downloading package punkt to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings

In [3]:
alias2entity = dataset.wikidata5m.load_alias("./entity.txt.gz")

In [4]:
print(entity_embeddings[entity2id[alias2entity["steve jobs"]]])

[-0.14506742 -0.12522382 -0.20576426 -0.24067238  0.5879003  -0.37743843
 -0.21119453  0.31490594  0.3750396   0.15038064 -0.22792946 -0.23479481
 -0.18943842 -0.31884384 -0.1297822  -0.16294236  0.20854223  0.24055439
  0.21060887 -0.31041607  0.02344835  0.2064475   0.23985358 -0.41276288
 -0.11470509  0.15619455  0.3971171   0.27328852 -0.20748287 -0.30876884
 -0.22771911 -0.30629387 -0.0895431  -0.17742126 -0.23951218  0.26153737
  0.21711522  0.36986688  0.50231826  0.2374765  -0.26459837 -0.22913334
  0.34746253  0.24404152 -0.18217663  0.08532561 -0.29877698  0.25021964
  0.11043487  0.27521297  0.21867076  0.23033242 -0.17963171  0.2614267
 -0.3470688   0.41214    -0.3862682   0.19821905  0.07231293 -0.42387047
  0.4534364  -0.2984923  -0.13255991 -0.09870375  0.3029415   0.1755728
 -0.19057822  0.34490088  0.4965025  -0.34853256  0.2103244  -0.17634982
  0.33285713  0.06456072  0.3649903   0.09194289 -0.247378    0.26242307
 -0.12918861 -0.29517058 -0.17388667  0.2873254   0.1

In [5]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 117.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
nlp = spacy.load('en_core_web_sm') 
sentence = "Who invented machine learning, was it Steve Jobs"
doc = nlp(sentence) 
print(doc)
for ent in doc.ents: 
    print(ent.text, ent.label_) 

Who invented machine learning, was it Steve Jobs
Steve Jobs PERSON


In [7]:
def clean_text(text):
    # Extract named entities
    nlp = spacy.load('en_core_web_sm') 
    named_entities = nlp(text).ents

    # Tokenize the text
    words = word_tokenize(text)

    # Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Part-of-speech tagging
    tagged_words = pos_tag(words)

    cleaned_text = ' '.join(words).lower()
    
    return cleaned_text, named_entities

# Example usage:
text = "Who invented machine learning, was it Steve Jobs?"
cleaned_text, named_entities = clean_text(text)
print("Cleaned Text:", cleaned_text)
print("Named Entities:", named_entities)

Cleaned Text: invented machine learning steve jobs
Named Entities: (Steve Jobs,)


In [8]:
def query_to_kg_embeddings(query):
    wiki_embeddings = []
    cleaned_text, named_entities = clean_text(query)
    # print(cleaned_text)
    for ne in named_entities: 
        try:
            #print(type(ne.orth_), ne.orth_.lower())
            ne = ne.orth_.lower()
            # print(ne, type(ne))
            # print(ne == "steve jobs")
            wiki_embeddings.append([ne, entity_embeddings[entity2id[alias2entity["steve jobs"]]]])
            cleaned_text = cleaned_text.replace(ne, "")
        except Exception as e:
            print("KeyError: ", e)


    return wiki_embeddings

In [9]:
print(query_to_kg_embeddings("Who invented calculus, was it Steve Jobs?"))

[['steve jobs', array([-0.14506742, -0.12522382, -0.20576426, -0.24067238,  0.5879003 ,
       -0.37743843, -0.21119453,  0.31490594,  0.3750396 ,  0.15038064,
       -0.22792946, -0.23479481, -0.18943842, -0.31884384, -0.1297822 ,
       -0.16294236,  0.20854223,  0.24055439,  0.21060887, -0.31041607,
        0.02344835,  0.2064475 ,  0.23985358, -0.41276288, -0.11470509,
        0.15619455,  0.3971171 ,  0.27328852, -0.20748287, -0.30876884,
       -0.22771911, -0.30629387, -0.0895431 , -0.17742126, -0.23951218,
        0.26153737,  0.21711522,  0.36986688,  0.50231826,  0.2374765 ,
       -0.26459837, -0.22913334,  0.34746253,  0.24404152, -0.18217663,
        0.08532561, -0.29877698,  0.25021964,  0.11043487,  0.27521297,
        0.21867076,  0.23033242, -0.17963171,  0.2614267 , -0.3470688 ,
        0.41214   , -0.3862682 ,  0.19821905,  0.07231293, -0.42387047,
        0.4534364 , -0.2984923 , -0.13255991, -0.09870375,  0.3029415 ,
        0.1755728 , -0.19057822,  0.34490088,  0

In [10]:
import os
halueval_path = os.path.join(os.getcwd(), "submodules", "HaluEval", "data", "qa_data.json")

In [11]:
halueval_path

'/var/home/zaristei/repos/KG-LLM-Hallucination/submodules/HaluEval/data/qa_data.json'

In [12]:
import json
with(open(halueval_path, "r")) as f:
    data_lst = f.readlines()

data_lst = [json.loads(data) for data in data_lst]

In [13]:
len(data_lst)

10000

In [14]:
data_lst[0]

{'knowledge': "Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman's magazine published by Bauer Media Group in the USA.",
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'right_answer': "Arthur's Magazine",
 'hallucinated_answer': 'First for Women was started first.'}

In [15]:
# RELEVANT_KEYS = list(data_lst[0].keys())
RELEVANT_KEYS = ['knowledge', 'question']

In [16]:
import tqdm

In [29]:
def process_data(data):
    result = dict()
    for k in RELEVANT_KEYS:
        result[k] = {i[0]: i[1] for i in query_to_kg_embeddings(data[k])}
        
    return result

In [30]:
processed_data = [process_data(i) for i in data_lst[:10]]

In [33]:
list(processed_data[0]['knowledge'].keys())

["arthur's magazine",
 'american',
 'philadelphia',
 'the 19th century',
 'first',
 'bauer media group',
 'usa']

In [42]:
from tqdm.contrib.concurrent import process_map
import multiprocessing as mp
import time

In [43]:
start = time.time()
result = process_map(process_data, data_lst, max_workers=mp.cpu_count())
end = time.time()
print(end-start)

/tmp/ipykernel_1185607/2783135458.py:2: TqdmWarning: Iterable length 10000 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  result = process_map(process_data, data_lst, max_workers=mp.cpu_count())


  0%|          | 0/10000 [00:00<?, ?it/s]

413.7066173553467


In [45]:
s = 0
for i in result:
    s += len(i['knowledge'].keys())
print(s / len(result))

10.1511


In [60]:
pickle.dump(result, open("processed_data.pkl", "wb"))

In [61]:
# Use these only if the pickle file doesn't work for whatever reason

In [50]:
reformatted_result = {i: list() for i in RELEVANT_KEYS}
for item in result:
    for k, vertices in item.items():
        reformatted_result[k].append({natural_language_entity: embedding.tolist() for natural_language_entity, embedding in vertices.items()})

In [59]:
with open("processed_data.json", "w") as f:
    json.dump(reformatted_result, f)